In [2]:
from data_tools import parse_boxes_from_json
from visualize import load_boxes
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import json

In [3]:
txt_path = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/txt/D0041-5370006.txt'
json_path = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/json/D0041-5370006.json'
im_path = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/images/D0041-5370006.tiff'

In [4]:
dir(json)

['JSONDecodeError',
 'JSONDecoder',
 'JSONEncoder',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_default_decoder',
 '_default_encoder',
 'codecs',
 'decoder',
 'detect_encoding',
 'dump',
 'dumps',
 'encoder',
 'load',
 'loads',
 'scanner']

In [6]:
with open(json_path, 'w') as jsf:
    print('hello')

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/json/D0041-5370006.json'